In [13]:
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName('Missing').getOrCreate()
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression

In [2]:
training = spark.read.csv('sample1.csv', header=True, inferSchema=True)

In [3]:
training.show()

+--------+---+----------+------+
|    Name|age|Experience|Salary|
+--------+---+----------+------+
|    jack| 31|        10| 30000|
|    alex| 30|         8| 25000|
|caroline| 29|         4| 20000|
|    paul| 24|         3| 20000|
|  sandra| 21|         1| 15000|
|casandra| 23|         2| 18000|
+--------+---+----------+------+



In [4]:
training.printSchema()

root
 |-- Name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- Experience: integer (nullable = true)
 |-- Salary: integer (nullable = true)



In [5]:
training.columns

['Name', 'age', 'Experience', 'Salary']

In [ ]:
# [Age,Experience]----> new feature--->independent feature

In [7]:
featureassembler = VectorAssembler(inputCols=['age','Experience'], outputCol='Independent Features')

In [8]:
output = featureassembler.transform(training)

In [9]:
output.show()

+--------+---+----------+------+--------------------+
|    Name|age|Experience|Salary|Independent Features|
+--------+---+----------+------+--------------------+
|    jack| 31|        10| 30000|         [31.0,10.0]|
|    alex| 30|         8| 25000|          [30.0,8.0]|
|caroline| 29|         4| 20000|          [29.0,4.0]|
|    paul| 24|         3| 20000|          [24.0,3.0]|
|  sandra| 21|         1| 15000|          [21.0,1.0]|
|casandra| 23|         2| 18000|          [23.0,2.0]|
+--------+---+----------+------+--------------------+



In [10]:
output.columns

['Name', 'age', 'Experience', 'Salary', 'Independent Features']

In [11]:
finalized_data = output.select('Independent Features','Salary')

In [12]:
finalized_data.show()

+--------------------+------+
|Independent Features|Salary|
+--------------------+------+
|         [31.0,10.0]| 30000|
|          [30.0,8.0]| 25000|
|          [29.0,4.0]| 20000|
|          [24.0,3.0]| 20000|
|          [21.0,1.0]| 15000|
|          [23.0,2.0]| 18000|
+--------------------+------+



In [14]:
train_data, test_data = finalized_data.randomSplit([0.75, 0.25])
regressor = LinearRegression(featuresCol='Independent Features', labelCol='Salary')
regressor = regressor.fit(train_data)

In [15]:
regressor.coefficients

DenseVector([-714.2857, 3485.7143])

In [16]:
regressor.intercept

26857.142857139796

In [17]:
pred_results = regressor.evaluate(test_data)

In [18]:
pred_results.predictions.show()

+--------------------+------+-----------------+
|Independent Features|Salary|       prediction|
+--------------------+------+-----------------+
|          [30.0,8.0]| 25000|33314.28571428435|
|         [31.0,10.0]| 30000|39571.42857142653|
+--------------------+------+-----------------+



In [20]:
pred_results.meanAbsoluteError

8942.857142855439

In [21]:
pred_results.meanSquaredError

80369795.91833645